In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import re 
print(os.listdir("../input"))


# Any results you write to the current directory are saved as output.

['test.ft.txt.bz2', 'train.ft.txt.bz2']


In [ ]:
import bz2
train_file = bz2.BZ2File('../input/train.ft.txt.bz2')
test_file = bz2.BZ2File('../input/test.ft.txt.bz2')

train_lines = train_file.readlines()
test_lines = test_file.readlines()

In [ ]:
print(len(train_lines), len(test_lines))
#2 indicates a positive review and 1 indicates a negative review
print(train_lines[0])
print(train_lines[10])


In [ ]:
train_lines = [x.decode('utf-8') for x in train_lines]
test_lines = [x.decode('utf-8') for x in test_lines]

In [ ]:
import re
from tqdm import tqdm

def pre_process(lines):
    reviews = []
    labels = []
    for line in tqdm(lines):
#         parse the review and label
        line = line.split('__label__')[1] 
        label = line[0]
        review = line[1:]
        review = review.lower()
        review = review.split()
#         ps = PorterStemmer()
#         review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
        review = ' '.join(review)
#         sentence=temp[1:len(temp)]
#         sentence[len(sentence)-1]=sentence[len(sentence)-1][0:len(sentence[len(sentence)-1])-3]
        
        reviews.append(review)
        labels.append(label)
        
    return reviews, labels
        
reviews_train, labels_train = pre_process(train_lines)
reviews_test, labels_test = pre_process(test_lines)
    





In [ ]:
labels_train = np.array(labels_train)
labels_test = np.array(labels_test)

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(reviews_train)

In [ ]:
token_train = tokenizer.texts_to_sequences(reviews_train)
token_test = tokenizer.texts_to_sequences(reviews_test)

In [ ]:
maxlen = 128
from keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(token_train, maxlen=128, padding='post')
x_test = pad_sequences(token_test, maxlen=128, padding='post')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

embed_dim = 128
model = Sequential()
model.add(Embedding(1000, 64,  input_length = x_train.shape[1]))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv1D(32, 7, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(Conv1D(2, 1))
model.add(GlobalAveragePooling1D())
model.add(Activation('softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
print(labels_train.shape)
y_train = []
for l in labels_train:
    if l == '1':
        l = [0,1]
    else:
        l = [1,0]
    y_train.append(l)
y_train = np.array(y_train)
print(y_train)

In [ ]:
print(labels_test.shape)
print(labels_test)
y_test = []
for l in labels_test:
    if l == '1':
        l = [0,1]
    else:
        l = [1,0]
    y_test.append(l)
y_test = np.array(y_test)
print(y_test)

In [ ]:
size = 200000
model.fit(x_train, y_train, batch_size=2048, epochs=5, validation_split=0.1)

In [ ]:
model.evaluate (x_test, y_test)
